In [30]:
import numpy as np
from tenpy.networks.mps import MPS
from tenpy.models.hubbard import FermiHubbardModel
from tenpy.algorithms import dmrg

def run_dmrg_2d(Lx, Ly, U, t, mu, chi_max, sweeps, charge=0):
    # Set up model parameters for a 2D square lattice
    model_params = {
        'lattice': "Square",  # Specify the type of lattice
        'Lx': Lx, 'Ly': Ly,   # Dimensions of the lattice
        'U': U,
        't': t,
        'mu': mu,
        'bc_MPS': 'finite',
        'bc_y': 'open',     
        'bc_x': 'periodic'# Boundary condition in the y-direction
    }

    model = FermiHubbardModel(model_params)
    # Initialize with an alternating pattern of filled and empty sites appropriate for 2D
    initial_state = []
    for x in range(Lx):
        for y in range(Ly):
            if (x + y) % 2 == 0:
                initial_state.append('up')
            else:
                initial_state.append('down')

    # Depending on charge modify the last site
    if charge == 1:
        initial_state[-1] = 'full'
    elif charge == -1:
        initial_state[-1] = 'empty'

    psi = MPS.from_product_state(model.lat.mps_sites(), initial_state, bc='finite')

    dmrg_params = {
        'mixer': True,
        'trunc_params': {
            'chi_max': chi_max,
            'svd_min': 1.e-10
        },
        'max_sweeps': sweeps,
    }

    eng = dmrg.TwoSiteDMRGEngine(psi, model, dmrg_params)
    E0, psi = eng.run()
    return E0


In [ ]:
Lx = 4
Ly = 10
U = 10
t = 1
mu = 0
chi_max = 1000
sweeps = 40

run_dmrg_2d(Lx, Ly, U, t, mu, chi_max, sweeps)